In [2]:
import gradio as gr
import keras
import numpy as np

In [4]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    # Calculate true positives, true negatives, false positives and false negatives
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    # Calculate precision and recall
    # Adding epsilon (small value) to combat potential division by 0
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    # Calculate F1
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [5]:
model_tuned = keras.models.load_model('breast_cancer_effnet_tuned.h5', custom_objects={'f1':f1})
labels = ['No Cancer', 'Breast Cancer']

In [6]:
def predict(inp):
        image = np.expand_dims(inp, 0)
        pred = model_tuned(image)
        prediction = np.squeeze(pred)
        confidences = {labels[1*(prediction > 0.5)]: float(prediction)}
        return confidences

In [7]:
demo = gr.Interface(fn=predict, 
             inputs=gr.inputs.Image(type="pil"),
             outputs=gr.outputs.Label(num_top_classes=2),
             examples=[["example/batch_sample_0.png", "example/batch_sample_1.png"]],
             )

C:\Users\Denys\AppData\Local\Temp\ipykernel_25676\4258672949.py:2: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(type="pil"),
C:\Users\Denys\AppData\Local\Temp\ipykernel_25676\4258672949.py:2: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(type="pil"),
C:\Users\Denys\AppData\Local\Temp\ipykernel_25676\4258672949.py:3: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Label(num_top_classes=2),
C:\Users\Denys\AppData\Local\Temp\ipykernel_25676\4258672949.py:3: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  outputs=gr.outputs.Label(num_top_classes=2),


In [8]:
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
